<img src="https://www.push-it-thermalstorage.eu/wp-content/uploads/2023/11/newlofo.png" width="400" alt="Logo PUSH-IT" title="Logo PUSH-IT">

# Design a CSEM survey for monitoring an Aquifer Thermal Energy Storage (ATES) site (II)

## (II) Modelling with **emg3d**: 3D

We will use the expected subsurface stratigraphy and related electrical resistivities at the ATES site of TU Delft. 

For more information on the project, have a look at https://www.push-it-thermalstorage.eu/pilots/delft/.

**Adjust the input values to emg3d based on the survey setup you identified as most suitable.**

**Setup 1:** Horizontal bipole source combined with 300 m long horizontal receiver line recording horizontal electric field component.

**Setup 2:** Horizontal bipole source combined with 300 m long vertical receiver line (in borehole) recording vertical electric field component.

**Setup 3:** Vertical bipole source combined with 300 m long vertical receiver line (in borehole) recording vertical electric field component.

**Tasks:** 
- Generate layered background and layered target responses with **empymod**.
- Generate 3D layered and target responses with **emg3d** (optional for different hot plume diameters, max. 160 m). Hot plume is approximated to be cubic.
- Compare layered background and layered target, and 3D layered target responses.
- Look at differences between 3D layered and 3D target responses.


**Questions:**
- Would you change your survey setup based on the 3D simulation results?
- How large must the hot plume be such that you can detect it?

In [ ]:
# Uncomment on Google Colab
# %pip install emg3d matplotlib discretize ipympl

In [ ]:
import emg3d
import empymod
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

plt.style.use('bmh')

# Comment this out on Google Colab
%matplotlib widget

## Survey setup and layered resistivity model

To get a as coarse as possible 3D grid (for speed of computation reasons), we simplify a few things.

In [ ]:
# You can copy/pased your code snippets for your preferred survey setup from empymod_ATES.ipynb here.

# Source
# Having x-directed sources on x-edges helps to have a minimal source signature
source = [-190, -210, 0, 0, 0, 0] # x-directed bipole

# Source frequency
frequency = 1

# Receiver offsets
offsets = np.linspace(1, 300, 300)  # 300 m long receiver line

# receiver-array: x, y, z, azimuth, dip
receivers = (offsets, offsets*0, 0, 0, 0)           # x-directed, Ex response
#receivers = (offsets*0, offsets*0, -offsets, 0, 90)  # z-directed, Ez response

# Layer boundaries
# We idealize the depth model here a bit, to have cells of 10m in depth
depth = [0, -50, -80, -120, -190]

# Layer resistivities in Ohm.m
res_target = 13.1
resistivities_base = np.array([1e8, 52.0, 38.0, 26.0, 29.5, 17.5]) # base incl. air (lower res, because of 3D modelling)
resistivities_target = resistivities_base.copy()
resistivities_target[4] = res_target  # adjust target resistivity (second last), now: aquifer +50 °Celsius

## Layered model responses

Compute base and target responses with **empymod**.

In [ ]:
# define the input that stays the same for all models
inp = {
    'src': source,
    'rec': receivers,
    'depth': depth,
    'freqtime': frequency,
    'srcpts': 5,  # Models the finite-length source as 5 point sources with Gaussian Quadrature
    'htarg': {'pts_per_dec': -1},  # Faster computation
    'verb': 1,
    'strength': 1,
}
resp_E_base = empymod.bipole(res=resistivities_base, **inp)
resp_E_target = empymod.bipole(res=resistivities_target, **inp)

## 3D target response with emg3d

We use a cubic shape for the hot plume.

Maximum diameter with + 50 °C after two years: 160 m.

Create the mesh:

In [ ]:
grid = emg3d.construct_mesh(
    center=(0,0,0),                                 # Center of wanted grid
    frequency=frequency,                            # Frequency we will use the grid for
    properties=[25, 25, 1e8],                       # Reference resistivity
    domain=([-350, 350], [-350, 350], [-200, 0]),   # Domain in which we want precise results
    center_on_edge=True,
    min_width_limits = [20, 20, 10],
)
 
grid  # mesh info
# grid.plot_grid() # plotting the grid

### 3D model: Introduce layers, cubic hot plume anomaly and assign resistivities

In [ ]:
# Pre-allocate an array with number of cells corresponding to our grid
res = np.ones(grid.shape_cells)

# Layered base model: Fill in layered resistivities into our grid
dd = np.r_[np.inf, depth, -np.inf]
for i in range(len(resistivities_base)):
    res[:, :, (grid.cell_centers_z > dd[i+1]) & (grid.cell_centers_z <= dd[i])] = resistivities_base[i]

# Create the layered base model
model_base = emg3d.Model(grid, property_x=res, mapping='Resistivity')

# 3D target model
# Insert cubic hot plume in reservoir layer with 160 m diameter, centered at zero.
res3 = res.ravel('F')
xx = abs(grid.cell_centers[:, 0]) <= 80
yy = abs(grid.cell_centers[:, 1]) <= 80
zz = (grid.cell_centers[:, 2] > depth[-1]) & (grid.cell_centers[:, 2] <= depth[-2])
res3[xx*yy*zz] = res_target  # Target resistivity

# Create the target model
model_target = emg3d.Model(grid, property_x=res3, mapping='Resistivity')  # Insert traget resistivity in model

# Plot/QC the model with the 3D target
grid.plot_3d_slicer(
    model_target.property_x,
    pcolor_opts={'norm': LogNorm(vmin=10, vmax=100)},
    xlim=[-350, 350],
    ylim=[-350, 350],
    zlim=[-300, 20],
    zslice=-160,
)

### Generate the source field with a certain frequency

In [ ]:
# Example is x-directed dipole source
sfield = emg3d.fields.get_source_field(grid, source=source, frequency=frequency)

### Call **emg3d** to solve for the electric field components

In [ ]:
# Layered base
efield3D_lay = emg3d.solve(model_base, sfield, verb=2)
# 3D target
efield3D_target = emg3d.solve(model_target, sfield, verb=2)

### Extract the receiver responses

In [ ]:
# Layered base
resp_E_base_3D = efield3D_lay.get_receiver(receivers, 'linear')
# 3D target
resp_E_target_3D = efield3D_target.get_receiver(receivers, 'linear')

### Plot your results along the receiver line

In [ ]:
# Plot it
fig3, (ax31, ax32) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, constrained_layout=True)

fig3.suptitle("Base vs layered target vs 3D target")

ax31.set_title('Amplitude |E| (V/m)')
ax31.plot(offsets, resp_E_base.amp(), 'k', label='background')
ax31.plot(offsets, resp_E_target.amp(), 'C0-', label='1D target empymod')
ax31.plot(offsets, resp_E_base_3D.amp(), 'y--', label='background emg3d') # scale your emg3d amplitues by the source length
ax31.plot(offsets, resp_E_target_3D.amp(), 'C1--', label='3D target emg3d') # scale your emg3d amplitues by the source length
ax31.set_yscale('log')
ax31.legend()

ax32.set_title('Phase (E) (°)')
ax32.plot(offsets, resp_E_base.pha(deg=True), 'k')
ax32.plot(offsets, resp_E_target.pha(deg=True), 'C0-')
ax32.plot(offsets, resp_E_base_3D.pha(deg=True), 'y--')
ax32.plot(offsets, resp_E_target_3D.pha(deg=True), 'C1--')

### Plot the **emg3d** response in the entire domain

In [ ]:
# Set the field to analyze: surface: Ex; borehole: Ez
if receivers[-1] == 0:
    field, ftype = 'fx', 'Ex'
else:
    field, ftype = 'fz', 'Ez'

In [ ]:
grid.plot_3d_slicer(
    getattr(efield3D_target, field).ravel('F'),
    view='abs',
    v_type=ftype,
    pcolor_opts={'norm': LogNorm()},
)
fig = plt.gcf()
fig.suptitle(f'Target Model, |{ftype}| (V/m)')

Plot the **emg3d** response in the inner domain:

In [ ]:
grid.plot_3d_slicer(
    getattr(efield3D_target, field).ravel('F'),
    view='abs',
    v_type=ftype,
    pcolor_opts={'norm': LogNorm(vmin=1e-9, vmax=1e-2)},
    xlim=[-350, 350],
    ylim=[-350, 350],
    zlim=[-300, 20],
    zslice=-160,
)
fig = plt.gcf()
fig.suptitle(f'Target Model, |{ftype}| (V/m)')

Plot the relative **emg3d** response differences between layered background and 3D target in the inner domain:

In [ ]:
diff = abs(getattr(efield3D_target, field).ravel('F') - getattr(efield3D_lay, field).ravel('F'))

grid.plot_3d_slicer(
    diff,
    view='abs',
    v_type=ftype,
    pcolor_opts={'norm': LogNorm(vmin=1e-7, vmax=1e-4), 'cmap': 'plasma' },
    xlim=[-350, 350],
    ylim=[-350, 350],
    zlim=[-300, 20],
    zslice=-160,
)
fig = plt.gcf()
fig.suptitle(f'Target Model - Background, |{ftype}| (V/m)')

In [ ]:
emg3d.Report()